<a target="_blank" href="https://colab.research.google.com/github/michalis0/Cloud-and-Advanced-Analytics/blob/main/labs/06-Elastic/Autocomplete/Autocomplete.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## **Autocomplete with Elastic Search:**

In [1]:
# Download libraries
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.6 MB/s eta 0:00:00


In [4]:
URL_ENDPOINT =   "https://.........:443"
API_KEY = ""
INDEX_NAME = 'trending_videos_youtube_elastic_data'

In [6]:
from elasticsearch import Elasticsearch
# Initiate the connexion to the index
client = Elasticsearch(
  URL_ENDPOINT,
  api_key=API_KEY
)

In [22]:
# Check connexion
client.info()

ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': '407ca2e2bcc744a8b51be000a82096de', 'cluster_uuid': 'g-FHNu1dRO6cH2VKEu2hFg', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [24]:
# Search test
response = client.search(index="trending_videos_youtube_elastic_data", q="Pourquoi")
# Process search results
for hit in response['hits']['hits']:
    print(hit['_source']['title'], hit['_source']['videoDifficulty'])

PourQuoi un scooter est mieux qu'ne moto ? C2
⚡ Pourquoi le triphasé ? - HS - Monsieur Bidouille B1
Pourquoi faut-il éviter les hybrides rechargeables ? B2
Pourquoi le conflit Israël-Palestine est si important ? B1
Pourquoi et comment mémoriser du VOCABULAIRE | Français COURANT + sous-titres  et transcription PDF. B2


In [25]:
# print the entire response
response

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 3.4007745, 'hits': [{'_index': 'trending_videos_youtube_elastic_data', '_id': 'MFb9dI4Bb1Z5B_xfRFHr', '_score': 3.4007745, '_source': {'channel_title': 'The Choucroute Garage', 'videoDifficulty': 'C2', '@timestamp': '2022-09-09T00:00:00.000+02:00', 'category_id': 'Autos & Vehicles', 'publish_time': '2022-09-09', 'title': "PourQuoi un scooter est mieux qu'ne moto ?", 'views': 42000, 'video_id': 'oRi3EMoSckY', 'likes': 1000}}, {'_index': 'trending_videos_youtube_elastic_data', '_id': 'UVb9dI4Bb1Z5B_xfRFHr', '_score': 3.4007745, '_source': {'channel_title': 'Monsieur Bidouille', 'videoDifficulty': 'B1', '@timestamp': '2018-07-30T00:00:00.000+02:00', 'category_id': 'Science & Technology', 'publish_time': '2018-07-30', 'title': '⚡ Pourquoi le triphasé ? - HS - Monsieur Bidouille', 'views': 325000, 'video_id': 'J

In [28]:
# Get all the categories
aggregation_query = {
    "aggs": {
        "category_ids": {
            "terms": {
                "field": "category_id",
                "size": 100
            }
        }
    }
}

res = client.search(index="trending_videos_youtube_elastic_data", body=aggregation_query)

category_ids = [bucket['key'] for bucket in res['aggregations']['category_ids']['buckets']]
print(category_ids)

['Education', 'News & Politics', 'Science & Technology', 'Entertainment', 'Autos & Vehicles', 'Howto & Style', 'People & Blogs', 'Nonprofits & Activism', 'Travel & Events', 'Film & Animation', 'Sports']


In [41]:
# Search by boolean query
body = {
    "query": {
        "bool": {
            "must": [
                {"match": {"category_id": "Education"}}
            ],
            "filter": [
                {"range": {"likes": {"gte": 100000}}} # Greater of Equal to
            ]
        }
    }
}

response = client.search(index="trending_videos_youtube_elastic_data", body=body)

# Process search results
for hit in response['hits']['hits']:
    print(hit['_source']['title'], hit['_source']['videoDifficulty'],hit['_source']['likes'])

Cette vidéo t'explique le conflit historique entre Israël et Palestine B2 192000
Tu peux mieux parler français dans 30 jours B1 144000


In [33]:
# Search test
response = client.search(index="trending_videos_youtube_elastic_data", q="Pourquoi")
# Process search results
for hit in response['hits']['hits']:
    print(hit['_source'])

{'channel_title': 'The Choucroute Garage', 'videoDifficulty': 'C2', '@timestamp': '2022-09-09T00:00:00.000+02:00', 'category_id': 'Autos & Vehicles', 'publish_time': '2022-09-09', 'title': "PourQuoi un scooter est mieux qu'ne moto ?", 'views': 42000, 'video_id': 'oRi3EMoSckY', 'likes': 1000}
{'channel_title': 'Monsieur Bidouille', 'videoDifficulty': 'B1', '@timestamp': '2018-07-30T00:00:00.000+02:00', 'category_id': 'Science & Technology', 'publish_time': '2018-07-30', 'title': '⚡ Pourquoi le triphasé ? - HS - Monsieur Bidouille', 'views': 325000, 'video_id': 'JwOTA5a2mK0', 'likes': 17000}
{'channel_title': 'Max BLD', 'videoDifficulty': 'B2', '@timestamp': '2022-05-31T00:00:00.000+02:00', 'category_id': 'Autos & Vehicles', 'publish_time': '2022-05-31', 'title': 'Pourquoi faut-il éviter les hybrides rechargeables ?', 'views': 74000, 'video_id': '1TAU7MsTXKM', 'likes': 711}
{'channel_title': 'TAOQAN', 'videoDifficulty': 'B1', '@timestamp': '2023-10-20T00:00:00.000+02:00', 'category_id': 

In [12]:
# Function to perform autocomplete search
def autocomplete_search(query):
    body = {
        "query": {
            "match_phrase_prefix": {
                "title": {
                    "query": query,
                    "max_expansions": 10  # Adjust the number of expansions as needed
                }
            }
        },
        "sort": [
      {"views": {"order": "desc"}}  # Sort by view count
        ]
    }


    response = client.search(index=INDEX_NAME, body=body)
    suggestions = [hit['_source']['title'] for hit in response['hits']['hits']]
    num_hits = len(suggestions)
    return suggestions,num_hits


while True:
    search_query = input("Enter video title (type 'exit' to quit): ")
    print("-----------------------")
    if search_query.lower() == 'exit':
        break

    # Perform autocomplete search
    autocomplete_results,num_hits = autocomplete_search(search_query)

    print("Autocomplete Suggestions:")
    print("Number of results",num_hits)
    print("------------------------")
    for result in autocomplete_results:
        print(result)